# 🔥 KURE 기반 한국형 번아웃 감정 분류 모델

**모델 구조**: KURE 임베딩 (1024차원) + 분류 헤드 → 4개 번아웃 카테고리

**번아웃 카테고리**:
- 0: 정서적_고갈 (에너지 소진, 피로, 무력감)
- 1: 좌절_압박 (분노, 불만, 억울함)
- 2: 부정적_대인관계 (대인 갈등, 소외감)
- 3: 자기비하 (불안, 자책, 수치심)

## 1. 환경 설정

In [ ]:
# 패키지 설치
!pip install -q sentence-transformers transformers torch pandas scikit-learn tqdm

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from typing import List, Dict, Optional, Tuple
from tqdm.auto import tqdm
import json
import os

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Using device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. 번아웃 카테고리 정의

In [ ]:
# 번아웃 카테고리
BURNOUT_CATEGORIES = {
    0: "정서적_고갈",
    1: "좌절_압박",
    2: "부정적_대인관계",
    3: "자기비하"
}

# 감정 → 번아웃 매핑
EMOTION_TO_BURNOUT = {
    # 정서적 고갈 (0)
    "스트레스": 0, "무력함": 0, "공허함": 0, "답답함": 0,
    "부담": 0, "지루함": 0, "귀찮음": 0, "후회": 0, "외로움": 0,
    "피곤": 0, "지침": 0, "무기력": 0, "우울": 0, "슬픔": 0,

    # 좌절/압박 (1)
    "억울": 1, "불만": 1, "원망": 1, "분노": 1, "불평": 1,
    "불편함": 1, "불쾌": 1, "질투": 1, "당황": 1, "화남": 1, "짜증": 1,

    # 부정적 대인관계 (2)
    "갈등": 2, "배신": 2, "무시": 2, "소외": 2, "서운함": 2,

    # 자기비하 (3)
    "불안": 3, "걱정": 3, "초조": 3, "자괴감": 3, "죄책감": 3,
    "혼란함": 3, "두려움": 3, "부끄러움": 3, "자책": 3,
}

print("✓ 카테고리 정의 완료")

## 3. 모델 정의

In [ ]:
class KUREEmbeddingLayer(nn.Module):
    """KURE 임베딩 레이어 (1024차원)"""

    def __init__(self, model_name="nlpai-lab/KURE-v1", freeze_encoder=True, device=None):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        print(f"📥 Loading KURE model: {model_name}")
        print("   (첫 실행 시 ~2GB 다운로드, 몇 분 소요)")

        self.encoder = SentenceTransformer(model_name, device=self.device)
        self.embedding_dim = self.encoder.get_sentence_embedding_dimension()

        print(f"✓ Embedding dimension: {self.embedding_dim}")

        if freeze_encoder:
            for param in self.encoder.parameters():
                param.requires_grad = False
            print("✓ Encoder frozen")

    def forward(self, texts: List[str]) -> torch.Tensor:
        return self.encoder.encode(texts, convert_to_tensor=True, show_progress_bar=False)


class BurnoutClassificationHead(nn.Module):
    """분류 헤드"""

    def __init__(self, input_dim=1024, hidden_dim=256, num_classes=4, dropout=0.3):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, embeddings):
        return self.classifier(embeddings)


class KUREBurnoutClassifier(nn.Module):
    """KURE + 분류 헤드 전체 모델"""

    def __init__(self, model_name="nlpai-lab/KURE-v1", hidden_dim=256,
                 num_classes=4, dropout=0.3, freeze_encoder=True, device=None):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        self.embedding_layer = KUREEmbeddingLayer(
            model_name=model_name,
            freeze_encoder=freeze_encoder,
            device=self.device
        )

        self.classification_head = BurnoutClassificationHead(
            input_dim=self.embedding_layer.embedding_dim,
            hidden_dim=hidden_dim,
            num_classes=num_classes,
            dropout=dropout
        ).to(self.device)

        self.num_classes = num_classes

    def forward(self, texts: List[str]):
        embeddings = self.embedding_layer(texts)
        return self.classification_head(embeddings)

    def predict(self, texts: List[str]):
        self.eval()
        with torch.no_grad():
            logits = self.forward(texts)
            probs = F.softmax(logits, dim=-1)
            labels = torch.argmax(logits, dim=-1).cpu().tolist()
            categories = [BURNOUT_CATEGORIES[l] for l in labels]
        return labels, categories, probs

    def save_model(self, path):
        torch.save({
            'classification_head': self.classification_head.state_dict(),
            'num_classes': self.num_classes
        }, path)
        print(f"✓ Saved to {path}")

    def load_model(self, path):
        ckpt = torch.load(path, map_location=self.device)
        self.classification_head.load_state_dict(ckpt['classification_head'])
        print(f"✓ Loaded from {path}")

print("✓ 모델 클래스 정의 완료")

## 4. 데이터셋 & 학습 함수

In [ ]:
class BurnoutDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {'text': self.texts[idx], 'label': self.labels[idx]}


def collate_fn(batch):
    texts = [item['text'] for item in batch]
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.long)
    return {'texts': texts, 'labels': labels}


def train_model(model, train_dataset, val_dataset=None, epochs=10,
                batch_size=16, learning_rate=2e-4):
    """모델 학습"""
    device = model.device

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_fn)
    val_loader = None
    if val_dataset:
        val_loader = DataLoader(val_dataset, batch_size=batch_size,
                                shuffle=False, collate_fn=collate_fn)

    optimizer = torch.optim.AdamW(model.classification_head.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    history = {'train_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in pbar:
            texts = batch['texts']
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(texts)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        train_acc = 100 * correct / total
        avg_loss = total_loss / len(train_loader)
        history['train_loss'].append(avg_loss)
        history['train_acc'].append(train_acc)

        # Validation
        val_acc = 0
        if val_loader:
            model.eval()
            val_correct, val_total = 0, 0
            with torch.no_grad():
                for batch in val_loader:
                    texts = batch['texts']
                    labels = batch['labels'].to(device)
                    logits = model(texts)
                    _, predicted = torch.max(logits, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
            val_acc = 100 * val_correct / val_total
            history['val_acc'].append(val_acc)

        print(f"  Loss: {avg_loss:.4f} | Train Acc: {train_acc:.1f}% | Val Acc: {val_acc:.1f}%")

    return model, history

print("✓ 데이터셋 & 학습 함수 정의 완료")

## 5. 샘플 데이터 생성

In [ ]:
def create_sample_data():
    """테스트용 샘플 데이터"""
    train_texts = [
        # 정서적 고갈 (0)
        "오늘도 야근했다. 너무 지친다.",
        "아무것도 하기 싫고 무력하다.",
        "매일 반복되는 일상이 지겹다.",
        "에너지가 바닥났다.",
        "쉬어도 피로가 안 풀린다.",
        "일하기 싫은데 해야 한다.",
        "번아웃이 온 것 같다.",
        "몸도 마음도 지쳐있다.",

        # 좌절/압박 (1)
        "상사에게 또 깨졌다. 억울하다.",
        "노력해도 인정받지 못한다.",
        "불합리한 지시에 화가 난다.",
        "성과를 가로채여서 분하다.",
        "압박감에 숨이 막힌다.",
        "진짜 짜증나는 하루였다.",
        "왜 나만 이런 일을 당해야 하지.",
        "불공평해서 미칠 것 같다.",

        # 부정적 대인관계 (2)
        "동료들이 나를 따돌리는 것 같다.",
        "팀 분위기가 너무 안 좋다.",
        "사람들과 대화하기 싫다.",
        "직장 내 갈등으로 힘들다.",
        "상사와의 관계가 최악이다.",
        "아무도 내 편이 없는 것 같다.",
        "사내 정치에 지쳤다.",
        "동료에게 배신당한 기분이다.",

        # 자기비하 (3)
        "나는 능력이 없는 것 같다.",
        "실수할까봐 불안하다.",
        "다른 사람들보다 못한 것 같다.",
        "자신감이 바닥이다.",
        "이런 나 자신이 부끄럽다.",
        "내가 잘할 수 있을까 걱정된다.",
        "자꾸 실수해서 자책하게 된다.",
        "나만 뒤처지는 것 같다.",
    ]

    train_labels = [0]*8 + [1]*8 + [2]*8 + [3]*8
    return train_texts, train_labels

train_texts, train_labels = create_sample_data()
print(f"✓ 샘플 데이터: {len(train_texts)}개")
print(f"  - 정서적_고갈: 8개")
print(f"  - 좌절_압박: 8개")
print(f"  - 부정적_대인관계: 8개")
print(f"  - 자기비하: 8개")

## 6. 모델 학습 🚀

In [ ]:
# 모델 초기화
print("=" * 50)
print("🚀 모델 초기화")
print("=" * 50)

model = KUREBurnoutClassifier(
    model_name="nlpai-lab/KURE-v1",
    hidden_dim=256,
    num_classes=4,
    dropout=0.3,
    freeze_encoder=True
)

In [ ]:
# 학습
print("\n" + "=" * 50)
print("📚 모델 학습")
print("=" * 50)

train_dataset = BurnoutDataset(train_texts, train_labels)

model, history = train_model(
    model=model,
    train_dataset=train_dataset,
    epochs=10,
    batch_size=8,
    learning_rate=1e-3
)

## 7. 예측 테스트

In [ ]:
print("\n" + "=" * 50)
print("🔮 예측 테스트")
print("=" * 50)

test_texts = [
    "오늘 정말 피곤하다. 아무것도 하기 싫어.",
    "상사가 또 무리한 요구를 했다. 열받아.",
    "팀원들이 나를 무시하는 것 같아 속상하다.",
    "발표를 앞두고 너무 불안하다. 잘할 수 있을까.",
    "회사 다니기 싫은데 돈은 벌어야 하고...",
]

labels, categories, probs = model.predict(test_texts)

for i, (text, cat, prob) in enumerate(zip(test_texts, categories, probs)):
    print(f"\n[{i+1}] {text}")
    print(f"    → 예측: {cat}")
    prob_dict = {name: f"{p:.1%}" for name, p in zip(BURNOUT_CATEGORIES.values(), prob.cpu().numpy())}
    print(f"    → 확률: {prob_dict}")

## 8. 모델 저장 (선택)

In [ ]:
# Google Drive 마운트 (선택)
# from google.colab import drive
# drive.mount('/content/drive')

# 모델 저장
# model.save_model('/content/drive/MyDrive/burnout_model.pt')

# 로컬 저장
model.save_model('burnout_model.pt')

## 9. 커스텀 테스트

In [ ]:
# 직접 문장 입력해서 테스트
my_text = "월요일 아침부터 야근 예고를 받았다. 벌써 지친다."  # @param {type:"string"}

labels, categories, probs = model.predict([my_text])

print(f"\n입력: {my_text}")
print(f"\n🏷️ 예측 카테고리: {categories[0]}")
print(f"\n📊 확률 분포:")
for name, p in zip(BURNOUT_CATEGORIES.values(), probs[0].cpu().numpy()):
    bar = "█" * int(p * 20)
    print(f"  {name}: {bar} {p:.1%}")